In [1]:
import pandas as pd
import json
import myql
from yahoo_oauth import OAuth1

In [44]:
cols = ['Name', 'Team', 'GP', 'G', 'A', 'Pts', 'PIM', 'PPP', 'SOG', 'FW', 'Hits']
# reads 2012 data
df12 = pd.read_csv('Stats/Fantasy Hockey Stats - 2012.csv')
df12['PPP'] = df12.PPG+df12.PPA
df12 = df12[cols]
# reads 2013 data
df13 = pd.read_csv('Stats/Fantasy Hockey Stats - 2013.csv')
df13['PPP'] = df13.PPG+df13.PPA
df13 = df13[cols]
# reads 2014 data
df14 = pd.read_csv('Stats/Fantasy Hockey Stats - 2014.csv')
df14['PPP'] = df14.PPG+df14.PPA
df14 = df14[cols]
# reads 2015 data
df15 = pd.read_html(
    "http://sports.yahoo.com/nhl/stats/byposition?pos=C%2CRW%2CLW%2CD&sort=14&conference=NHL&year=season_2015",
    match = 'Name', flavor = 'bs4', header = 0)[0]
df15['PPP'] = df15.PPG+df15.PPA
df15 = df15[cols]

In [45]:
# reads goalie data
g12 = pd.read_csv('Stats/Fantasy Hockey Stats - g12.csv')
g13 = pd.read_csv('Stats/Fantasy Hockey Stats - g13.csv')
g14 = pd.read_csv('Stats/Fantasy Hockey Stats - g14.csv')
g15 = pd.read_html('http://sports.yahoo.com/nhl/stats/byposition?pos=G&sort=102&conference=NHL&year=season_2015',
    match = 'Name', flavor = 'bs4', header = 0)[0]

In [46]:
# merges data
df = df15.append(df12)
df = df.append(df13)
df = df.append(df14)
df = df.groupby('Name').sum()
gf = g15.append(g12)
gf = gf.append(g13)
gf = gf.append(g14)
gf = gf.groupby('Name').sum()

In [47]:
# creates skater average stats
df['G_avg'] = df['G']/df['GP']
df['A_avg'] = df['A']/df['GP']
df['P_avg'] = df['Pts']/df['GP']
df['PIM_avg'] = df['PIM']/df['GP']
df['PPP_avg'] = df['PPP']/df['GP']
df['SOG_avg'] = df['SOG']/df['GP']
df['FW_avg'] = df['FW']/df['GP']
df['HITS_avg'] = df['Hits']/df['GP']
avg_cols = ['GP', 'G_avg', 'A_avg', 'P_avg', 'PIM_avg', 'PPP_avg', 
            'SOG_avg', 'FW_avg', 'HITS_avg']
skaters = df[avg_cols]

In [48]:
# creates goalie average stats
gf_cols = ['GP', 'MIN', 'SV', 'SA', 'GA']
gf = gf[gf_cols]
gf['SV%'] = gf['SV']/gf['SA']
gf['GAA'] = gf['GA']/ (gf['MIN'] / 60)
gf['POS'] = 'G'
gf['SV_avg'] = gf['SV']/gf['GP']
gfGP = gf['GP']
gf = gf[['GAA', 'SV_avg', 'SV%']]
gf['GAA'] = gf['GAA'] * -1

In [49]:
# adds positions
file = pd.read_csv('Stats/Fantasy Hockey Stats - Defense.csv')
defense = df[avg_cols][df.index.isin(file['Name'])]
forwards = df[avg_cols][~df.index.isin(file['Name'])]

# limits to players active this season
defense = defense[defense.index.isin(df15['Name'])]
forwards = forwards[forwards.index.isin(df15['Name'])]
goalies = gf[gf.index.isin(g15['Name'])]

In [50]:
# strips and saves games played data
skatersGP = skaters.GP
skaters = skaters[skaters.columns[1:]]
defenseGP = defense.GP
defense = defense[defense.columns[1:]]
forwardsGP = forwards.GP
forwards = forwards[forwards.columns[1:]]

In [51]:
# normalizes data

defense_norm = (defense - defense.mean()) / defense.std()
defense_norm['Score'] = defense_norm.sum(axis=1)
defense_norm['GP'] = defenseGP

forwards_norm = (forwards - forwards.mean()) / forwards.std()
forwards_norm['Score'] = forwards_norm.sum(axis=1)
forwards_norm['GP'] = forwardsGP

goalies_norm = (goalies - goalies.mean()) / goalies.std()
goalies_norm['Score'] = goalies_norm.sum(axis=1)
goalies_norm['GP'] = gfGP

In [52]:
# ranks by position

defense = defense_norm.sort_values('Score', ascending=False)
rank = list(range(1,len(defense)+1))
defense['PRank'] = rank
defense['POS'] = 'D'
defense = defense[['PRank', 'Score', 'POS', 'GP','G_avg', 'A_avg', 'P_avg', 
                   'PIM_avg', 'PPP_avg', 'SOG_avg', 'FW_avg', 'HITS_avg']]

forwards = forwards_norm.sort_values('Score', ascending=False)
rank = list(range(1,len(forwards)+1))
forwards['PRank'] = rank
forwards['POS'] = 'F'
forwards = forwards[['PRank', 'Score', 'POS', 'GP','G_avg', 'A_avg', 'P_avg',
                     'PIM_avg', 'PPP_avg', 'SOG_avg', 'FW_avg', 'HITS_avg']]

goalies = goalies_norm.sort_values('Score', ascending=False)
rank = list(range(1,len(goalies)+1))
goalies['PRank'] = rank
goalies['POS'] = 'G'
goalies = goalies[['PRank', 'Score', 'POS', 'GP','GAA', 'SV_avg', 'SV%']]

In [53]:
# merges positions and adds overall rank
allskaters = defense.append(forwards)
allskaters = allskaters.append(goalies)
allskaters = allskaters.sort_values('Score', ascending = False)
ORank = list(range(1, len(allskaters)+1))
allskaters['ORank'] = ORank
allskaters['Team'] = 'FA'
allskaters = allskaters[['ORank', 'PRank', 'Score', 'Team', 'POS', 'GP','G_avg', 'A_avg', 
                         'P_avg', 'PIM_avg', 'PPP_avg', 'SOG_avg', 'FW_avg', 'HITS_avg',
                        'GAA', 'SV_avg', 'SV%']]

In [54]:
oauth = OAuth1(None, None, from_file='credentials.json')
yql = myql.MYQL(oauth = oauth)
response = yql.raw_query("select * from fantasysports.teams.roster where league_key='352.l.63158'")
data = response.json()

with open('rosters.json', 'w') as outfile:
    json.dump(data, outfile)

[2016-01-02 12:09:29,489 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2016-01-02 12:09:29,491 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 412.8947651386261
[2016-01-02 12:09:29,491 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID
[2016-01-02 12:09:29,493 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 412.8968460559845
[2016-01-02 12:09:29,493 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [55]:
# creates dict of team rosters
with open('rosters.json') as data_file:    
    data = json.load(data_file)
teams = []
for num in range(0,14):
    teams.append(data['query']['results']['team'][num]['name'])
count = 0
rosters = {}
for team in teams:
    rosters[team] = []
    for num in range(0, int(data['query']['results']['team'][count]
                            ['roster']['players']['count'])):
        rosters[team].append(data['query']['results']['team'][count]
                             ['roster']['players']['player'][num]['name']['full'])
    count +=1

In [56]:
# creates dataframe of each team and adds team to allskaters
Game_of_Doans = allskaters[allskaters.index.isin(rosters['Game of Doans'])]
Horsedick = allskaters[allskaters.index.isin(rosters['Horsedick.mpeg'])]
Fists_of_Fleury = allskaters[allskaters.index.isin(rosters['Fists of Fleury'])]
Munson_Munson = allskaters[allskaters.index.isin(rosters['Munson Munson'])]
The_Ants = allskaters[allskaters.index.isin(rosters['The Ants'])]
Ricks_Red_Wingels = allskaters[allskaters.index.isin(rosters["Rick's Red Wingels"])]
CamelToews = allskaters[allskaters.index.isin(rosters['CamelToews'])]
Sarahs_Super_Team = allskaters[allskaters.index.isin(rosters["Sarah's Super Team"])]
Nordic_Supermen = allskaters[allskaters.index.isin(rosters['Nordic Supermen'])]
DeKeyser = allskaters[allskaters.index.isin(rosters['DeKeyser Söze'])]
Shiroi_Kabocha = allskaters[allskaters.index.isin(rosters['Shiroi Kabocha'])]
Brooklyn_Solar_Bears = allskaters[allskaters.index.isin(rosters['Brooklyn Solar Bears'])]
STEPANWOLF = allskaters[allskaters.index.isin(rosters['STEPANWOLF'])]
Caress_of_Krieger = allskaters[allskaters.index.isin(rosters['Caress of Krieger'])]

allskaters['Team'][allskaters.index.isin(Game_of_Doans.index)] = 'Game of Doans'
allskaters['Team'][allskaters.index.isin(Horsedick.index)] = 'Horsedick.mpeg'
allskaters['Team'][allskaters.index.isin(Fists_of_Fleury.index)] = 'Fists of Fleury'
allskaters['Team'][allskaters.index.isin(Munson_Munson.index)] = 'Munson Munson'
allskaters['Team'][allskaters.index.isin(The_Ants.index)] = 'The Ants'
allskaters['Team'][allskaters.index.isin(Ricks_Red_Wingels.index)] = "Rick's Red Wingels"
allskaters['Team'][allskaters.index.isin(CamelToews.index)] = 'CamelToews'
allskaters['Team'][allskaters.index.isin(Sarahs_Super_Team.index)] = "Sarah's Super Team"
allskaters['Team'][allskaters.index.isin(Nordic_Supermen.index)] = 'Nordic Supermen'
allskaters['Team'][allskaters.index.isin(DeKeyser.index)] = 'DeKeyser Söze'
allskaters['Team'][allskaters.index.isin(Shiroi_Kabocha.index)] = 'Shiroi Kabocha'
allskaters['Team'][allskaters.index.isin(Brooklyn_Solar_Bears.index)] = 'Brooklyn Solar Bears'
allskaters['Team'][allskaters.index.isin(STEPANWOLF.index)] = 'STEPANWOLF'
allskaters['Team'][allskaters.index.isin(Caress_of_Krieger.index)] = 'Caress of Krieger'

Game_of_Doans = allskaters[allskaters.index.isin(rosters['Game of Doans'])]
Horsedick = allskaters[allskaters.index.isin(rosters['Horsedick.mpeg'])]
Fists_of_Fleury = allskaters[allskaters.index.isin(rosters['Fists of Fleury'])]
Munson_Munson = allskaters[allskaters.index.isin(rosters['Munson Munson'])]
The_Ants = allskaters[allskaters.index.isin(rosters['The Ants'])]
Ricks_Red_Wingels = allskaters[allskaters.index.isin(rosters["Rick's Red Wingels"])]
CamelToews = allskaters[allskaters.index.isin(rosters['CamelToews'])]
Sarahs_Super_Team = allskaters[allskaters.index.isin(rosters["Sarah's Super Team"])]
Nordic_Supermen = allskaters[allskaters.index.isin(rosters['Nordic Supermen'])]
DeKeyser = allskaters[allskaters.index.isin(rosters['DeKeyser Söze'])]
Shiroi_Kabocha = allskaters[allskaters.index.isin(rosters['Shiroi Kabocha'])]
Brooklyn_Solar_Bears = allskaters[allskaters.index.isin(rosters['Brooklyn Solar Bears'])]
STEPANWOLF = allskaters[allskaters.index.isin(rosters['STEPANWOLF'])]
Caress_of_Krieger = allskaters[allskaters.index.isin(rosters['Caress of Krieger'])]

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [57]:
def lookup(*args):
    players = list(args)
    return(allskaters[allskaters.index.isin(players)])

def value(player):
    return lookup(player).append(Game_of_Doans[Game_of_Doans['Score'] 
                                               < list(lookup(player)['Score'])])
def compare(team):
    compare = allskaters.groupby('Team').sum()
    compare = compare.drop('FA')
    compare = compare[['G_avg', 'A_avg', 'P_avg', 'PIM_avg', 'PPP_avg', 
         'SOG_avg', 'FW_avg', 'HITS_avg', 'GAA', 'SV_avg', 'SV%']]
    compare = compare[compare.index.isin([team])].append(
        compare[compare.index.isin(['Game of Doans'])])
    return(compare.diff()[1:])

def rankings():
    df = allskaters.groupby('Team').sum().drop('FA').ix[:,'G_avg':]
    for col in df:
        df[col] = df[col].rank(ascending = False)
    df['Rank'] = df.mean(axis=1)
    df['Rank'] = df['Rank'].rank()
    return(df[['Rank', 'G_avg', 'A_avg', 'P_avg', 'PIM_avg', 'PPP_avg', 
         'SOG_avg', 'FW_avg', 'HITS_avg', 'GAA', 'SV_avg', 'SV%']].sort_values('Rank'))

In [58]:
Game_of_Doans

,ORank,PRank,Score,Team,POS,GP,G_avg,A_avg,P_avg,PIM_avg,PPP_avg,SOG_avg,FW_avg,HITS_avg,GAA,SV_avg,SV%
Name,,,,,,,,,,,,,,,,,
Brent Burns,1,1,29.784369,Game of Doans,D,217,4.250429,1.910372,2.777859,0.347328,2.016967,3.581323,14.249204,0.650887,NaN,NaN,NaN
Steven Stamkos,11,6,13.193557,Game of Doans,F,205,3.480078,1.127016,2.285134,0.075857,2.790205,2.018956,1.547613,-0.131301,NaN,NaN,NaN
Pavel Datsyuk,21,15,10.741149,Game of Doans,F,178,1.712196,2.150650,2.148341,-0.744057,2.398321,1.180854,2.574944,-0.680098,NaN,NaN,NaN
Shayne Gostisbehere,22,7,10.576570,Game of Doans,D,22,3.915258,1.208665,2.143975,-1.032318,2.810774,1.002024,2.016011,-1.487819,NaN,NaN,NaN
Brandon Dubinsky,25,17,10.241078,Game of Doans,F,184,0.342552,1.556492,1.152365,1.663020,0.696083,0.624172,2.177818,2.028577,NaN,NaN,NaN
Dion Phaneuf,28,10,9.615535,Game of Doans,D,234,0.592429,1.171481,1.068614,2.755236,1.565346,1.004090,-0.133466,1.591805,NaN,NaN,NaN
Wayne Simmonds,46,31,8.537641,Game of Doans,F,238,1.590359,0.691802,1.157692,1.803927,2.198219,1.072549,-0.730036,0.753130,NaN,NaN,NaN
Tyler Bozak,72,51,7.216230,Game of Doans,F,218,1.130255,1.088859,1.205911,-0.355459,1.225992,0.011466,3.190854,-0.281649,NaN,NaN,NaN
Andrew Ladd,73,52,7.158611,Game of Doans,F,244,1.295014,1.358761,1.453028,0.401521,0.962482,1.213215,-0.653630,1.128220,NaN,NaN,NaN
